## Simulate transactions and explore different scenarios

In [10]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Imports
from metacash.labels import Labels
from metacash.metacash import MetaCash
from metacash.simulation.sim_transactions import SimulateAccount
from metacash.simulation.sim_transactions import SimulateTransactions, TimestampSampler, TransactionsSampler, \
    AmountSampler
from io import StringIO
import pandas as pd
import numpy as np

from metacash.dashboards.account_overview import AccountOverview
from metacash.dashboards.category_analysis import CategoryAnalysis


In [12]:
myf = MetaCash("datasets/local/complete.config.py")

ts_sampler = TimestampSampler.date_range(start="2020-01-01", periods=12, freq='MS')
sim = SimulateTransactions()
sim = SimulateTransactions()

noise = lambda v: v / 10 * np.random.normal(0, 1/2)

sim += TransactionsSampler(ts_sampler, AmountSampler(-1400.00, noise=noise), description="housing")
sim += TransactionsSampler(ts_sampler, AmountSampler(-850.00), description="health")
sim += TransactionsSampler(ts_sampler, AmountSampler(-200.00), description="food")
sim += TransactionsSampler(ts_sampler, AmountSampler(3000.00), description="income")
sim += TransactionsSampler(ts_sampler, AmountSampler(100.00), description="utilities")
sim += TransactionsSampler(ts_sampler, AmountSampler(100.00), description="transportation")
sim += TransactionsSampler(ts_sampler, AmountSampler(-100.00), description="personal")

transactions = sim.sample()

df_patterns = pd.read_csv(StringIO("""pattern;label
        housing;housing
        health;health
        food;food
        income;income
        """), sep=";")

patterns_dfs = {"category": df_patterns}
labels = Labels.load_df(transactions, patterns_dfs)
account = SimulateAccount(myf.config, "simulation", transactions, labels).account
print(account.df())

    timestamp  balance  amount currency     description type label.category
0  2020-01-01        0       0      EUR                   ib        unknown
1  2020-01-01  -1418.2 -1418.2      EUR         housing    t        housing
2  2020-01-01  -1318.2     100      EUR       utilities    t        unknown
3  2020-01-01  -1518.2    -200      EUR            food    t           food
4  2020-01-01  -1418.2     100      EUR  transportation    t        unknown
..        ...      ...     ...      ...             ...  ...            ...
81 2020-12-01  4573.67    -850      EUR          health    t         health
82 2020-12-01  7573.67    3000      EUR          income    t         income
83 2020-12-01  7673.67     100      EUR  transportation    t        unknown
84 2020-12-01  7573.67    -100      EUR        personal    t        unknown
85 2020-12-01  7573.67       0      EUR                   fb        unknown

[86 rows x 7 columns]


In [13]:
myf.accounts = { "sim": account }

In [14]:
myf.overview()

INFO    | simulation                              : [2020-01-01,2020-12-01] (86) +7,573.67


In [15]:
ao = AccountOverview(myf)
ao.display()


GridspecLayout(children=(Dropdown(description='Account:', layout=Layout(grid_area='widget001'), options=('sim'…

GridspecLayout(children=(Button(description='Overview', layout=Layout(grid_area='widget001', height='auto', wi…

Output()

In [17]:
# transactions as Pandas dataframe
account.df()

,timestamp,balance,amount,currency,description,type,label.category
0,2020-01-01,0,0,EUR,,ib,unknown
1,2020-01-01,-1418.2,-1418.2,EUR,housing,t,housing
2,2020-01-01,-1318.2,100,EUR,utilities,t,unknown
3,2020-01-01,-1518.2,-200,EUR,food,t,food
4,2020-01-01,-1418.2,100,EUR,transportation,t,unknown
...,...,...,...,...,...,...,...
81,2020-12-01,4573.67,-850,EUR,health,t,health
82,2020-12-01,7573.67,3000,EUR,income,t,income
83,2020-12-01,7673.67,100,EUR,transportation,t,unknown
84,2020-12-01,7573.67,-100,EUR,personal,t,unknown
